In [1]:
from google.colab import drive
drive_root = '/content/drive'
drive.mount(drive_root, force_remount=True)

Mounted at /content/drive


In [2]:
import sys
import os

project_root = os.path.join(drive_root, 'MyDrive/Colab Notebooks/cmpe540/final-project')
source_root = os.path.join(project_root, 'src')
sys.path.append(source_root)
data_folder_path = os.path.join(project_root, 'data')
raw_data_folder_path = os.path.join(data_folder_path, 'raw')
processed_data_folder_path = os.path.join(data_folder_path, 'processed')

In [3]:
import pandas as pd
input_file_path = os.path.join(processed_data_folder_path, "new_filtered_flight.csv")
flight = pd.read_csv(input_file_path)

In [4]:
# Filter for busiest route
flight_filtered = flight.copy()
flight_filtered = flight_filtered[flight_filtered['route'] == 'ATL-LAX']
flight_filtered = flight_filtered.drop(['route'], axis=1)

# Change date columns to datetime
for column in ['searchDate', 'flightDate']:
  flight_filtered[column] = pd.to_datetime(flight_filtered[column], format='%Y-%m-%d')

In [5]:
# Calculate Number (of) Days (to) Operation = NDO
flight_filtered['ndo'] = flight_filtered['flightDate'] - flight_filtered['searchDate']
flight_filtered = flight_filtered.drop(['searchDate'], axis=1)

# Remove rows which have ndo over 30, out-of-scope of this study
flight_filtered = flight_filtered[flight_filtered['ndo'] <= pd.Timedelta(days=30)]

In [6]:
# Arrival time creation
flight_filtered['arrivalEpoch'] = flight_filtered['segmentsArrivalTimeEpochSeconds'].apply(
    lambda x: int(x.split('||')[0])  # Convert to int after splitting
)
flight_filtered = flight_filtered.drop(['segmentsArrivalTimeEpochSeconds'], axis=1)

# Convert epoch to datetime
flight_filtered['arrivalDatetime'] = pd.to_datetime(flight_filtered['arrivalEpoch'], unit='s')
flight_filtered = flight_filtered.drop(['arrivalEpoch'], axis=1)

In [7]:
# Extract time (0.00, 24.00):24h from full date
flight_filtered['arrivalTimeFloat'] = flight_filtered['arrivalDatetime'].dt.hour + (flight_filtered['arrivalDatetime'].dt.minute / 60)
flight_filtered['arrivalTimeFloat'] = flight_filtered['arrivalTimeFloat'].round(2)
flight_filtered = flight_filtered.drop(['arrivalDatetime'], axis=1)

In [8]:
# Departure
flight_filtered['departureEpoch'] = flight_filtered['segmentsDepartureTimeEpochSeconds'].apply(
    lambda x: int(x.split('||')[0])  # Convert to int after splitting
)
flight_filtered = flight_filtered.drop(['segmentsDepartureTimeEpochSeconds'], axis=1)

# Convert epoch to datetime
flight_filtered['departureDatetime'] = pd.to_datetime(flight_filtered['departureEpoch'], unit='s')
flight_filtered = flight_filtered.drop(['departureEpoch'], axis=1)

# Extract "day-of-week" (1 for Monday, 7 for Sunday)
flight_filtered['dayOfWeek'] = flight_filtered['departureDatetime'].dt.isocalendar().day

# Extract "departure-time" as hours (0.00 to 24.00 format)
flight_filtered['departureTimeFloat'] = flight_filtered['departureDatetime'].dt.hour + (flight_filtered['departureDatetime'].dt.minute / 60)
flight_filtered['departureTimeFloat'] = flight_filtered['departureTimeFloat'].round(2)

# Remove rows where (possibly actual) departureDatetime does not match flightDate
# This is a workaround since this was the only available timestamped data, but for
# the scheduled times, it does not contain timestamp information
flight_filtered['temp_flightDate'] = flight_filtered['departureDatetime'].dt.date
flight_filtered[flight_filtered['temp_flightDate'] != flight_filtered['flightDate']]
flight_filtered = flight_filtered.drop(['departureDatetime', 'temp_flightDate'], axis=1)

In [9]:
# Create 'Departure date' feature: A certain date in the days investigated
flight_filtered = flight_filtered.sort_values(by=['flightDate', 'departureTimeFloat'], ascending=[True, True])

flight_filtered['NDepartureDate'] = (
    flight_filtered['flightDate']
    .rank(method='dense', ascending=True)
    .astype(int)
)

In [10]:
# P_i_j = P_flightDate_ndo
# Aggregate average fare for a given departure date
lut = flight_filtered.groupby(['NDepartureDate', 'ndo'])['baseFare'].mean().reset_index()
lut['baseFare'] = lut['baseFare'].round(2)

# Convert ndo to int for easy coordinate-based (i,j) access
lut['ndo'] = lut['ndo'].dt.days

In [11]:
# Pivot to create lut_matrix
lut_matrix = lut.pivot(
    index='NDepartureDate',  # Use NDepartureDate as the row index
    columns='ndo',           # Use ndo as the columns
    values='baseFare'        # Use baseFare for values
)

In [12]:
lut_matrix.columns

Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
      dtype='int64', name='ndo')

In [13]:
# Find the days we have ndo30 - ndo1 data of
unique_ndo_per_date = flight_filtered.groupby('flightDate')['ndo'].nunique().reset_index()
unique_ndo_per_date.rename(columns={'ndo': 'uniqueNdoValues'}, inplace=True)
eligible_dates = unique_ndo_per_date[unique_ndo_per_date['uniqueNdoValues'] >= 30]['flightDate'].tolist()
flight_filtered = flight_filtered[flight_filtered['flightDate'].isin(eligible_dates)]

In [14]:
import numpy as np

# Function to compute b1 to b30 for a given i and j
def compute_b_values(i, j, lut_matrix, max_b=30):
    b_values = []
    if i - 1 in lut_matrix.index:  # Check if previous row exists
        row_prev = lut_matrix.loc[i - 1]
        b_values += row_prev.iloc[max(0, j - max_b):j].tolist()

    if len(b_values) < max_b and i in lut_matrix.index:  # Fill remaining from current row
        row_curr = lut_matrix.loc[i]
        b_values += row_curr.iloc[:max_b - len(b_values)].tolist()

    # Pad with NaN if not enough values
    return b_values[:max_b] + [np.nan] * (max_b - len(b_values))

In [ ]:
# Add b1 to b30 as new columns in `flight_filtered`
flight_filtered['ndo'] = flight_filtered['ndo'].dt.days

# i : Departure date, j: ndo
for idx, row in flight_filtered.iterrows():
    i = row['NDepartureDate']
    j = row['ndo']
    b_values = compute_b_values(i, j, lut_matrix)
    for b_idx, b_val in enumerate(b_values, start=1):
        flight_filtered.at[idx, f'b{b_idx}'] = b_val

In [26]:
flight_filtered = flight_filtered[~flight_filtered.isna().any(axis=1)]

,flightDate,isNonStop,baseFare,ndo,arrivalTimeFloat,dayOfWeek,departureTimeFloat,NDepartureDate,b1,b2,...,b21,b22,b23,b24,b25,b26,b27,b28,b29,b30
1146592,2022-06-24,False,543.26,4,4.25,6,0.18,69,410.71,418.15,...,410.09,396.45,413.92,412.74,400.62,411.22,418.69,426.56,389.87,398.26
1164829,2022-06-24,False,543.26,3,4.25,6,0.18,69,410.71,418.15,...,396.45,413.92,412.74,400.62,411.22,418.69,426.56,389.87,398.26,412.31
1185797,2022-06-24,False,543.26,2,4.25,6,0.18,69,410.71,418.15,...,413.92,412.74,400.62,411.22,418.69,426.56,389.87,398.26,412.31,400.38
1205712,2022-06-24,False,543.26,1,4.25,6,0.18,69,410.71,434.18,...,412.74,400.62,411.22,418.69,426.56,389.87,398.26,412.31,400.38,483.86
675605,2022-06-24,True,422.33,29,5.45,6,0.82,69,410.71,418.15,...,396.60,383.54,403.28,400.60,400.44,388.45,392.58,402.07,406.80,434.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2476155,2022-08-31,False,322.79,2,4.75,3,23.53,137,354.48,316.60,...,228.74,222.11,170.58,162.03,166.13,173.05,174.80,177.48,188.67,189.99
2476158,2022-08-31,False,322.79,2,4.75,3,23.53,137,354.48,316.60,...,228.74,222.11,170.58,162.03,166.13,173.05,174.80,177.48,188.67,189.99
2476482,2022-08-31,False,322.79,1,4.75,3,23.53,137,354.48,351.19,...,222.11,170.58,162.03,166.13,173.05,174.80,177.48,188.67,189.99,190.16
2476485,2022-08-31,False,322.79,1,4.75,3,23.53,137,354.48,351.19,...,222.11,170.58,162.03,166.13,173.05,174.80,177.48,188.67,189.99,190.16


In [25]:
# Define the output file path
output_file_path = os.path.join(processed_data_folder_path, "train_data.csv")

# Save the DataFrame to a CSV file
flight_filtered.to_csv(output_file_path, index=False)